In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_current
CLUSTER BY (first_sale_stock_day, business_unit, store_id, product_id)
AS
SELECT 
    IFNULL(e.store_id, a.store_id) store_id,
    IF(e.store_id IS NOT NULL, '6000', business_unit) AS business_unit,
    b.product_id,
    b.mch1_id,
    b.mch2_id,
    b.mch3_id,
    b.mch4_id,
    b.mch5_id,
    b.mch6_id,
    c.city,
    c.addr_numbr,
    MIN(IF(revenue.base_sale_qty > 0 OR stock.closing_stock_quantity > 0, calday, NULL)) first_sale_stock_day,
    MIN(IF((IFNULL(revenue.base_sale_qty,0) != 0 OR IFNULL(stock.closing_stock_quantity,0) != 0) AND calday >= CAST((CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS DATE) - 90, calday, NULL)) first_sale_stock_90day,
    MIN(calday) first_issue_fact_day
FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily a
JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b USING (product_id)
JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c using (store_id)
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config e ON a.store_id = e.ref_fc_store
WHERE calday >= '2019-10-01'
GROUP BY 1,2,3,4,5,6,7,8,9,10,11
""")